In [1]:
from selenium import webdriver
from time import sleep
from PIL import Image
from pytesseract import image_to_string
import numpy as np
import argparse
import imutils
import glob
import cv2

In [2]:
class Bot:
    def __init__(self):
        self.driver = webdriver.Chrome()
        
#         self.driver.get('https://www.sberbank.ru/ru/person')
#         self.driver.save_screenshot('sber.png')
        self.count = 1
#         self.driver.get('https://codepunch.com/dnfeed/')
#         self.driver.save_screenshot('test.png')
        #self.driver.get('https://habr.com/ru/post/210238/')
        #self.driver.save_screenshot('habr.png')
        #self.driver.get('http://sber-opros.pro/questions-are-loaded.php')
        #self.driver.save_screenshot('sber_opros_screenshot.png')
        
    def read(self, adress):
        self.driver.get(adress)
        name = str(self.count) + '.png'
        self.driver.save_screenshot(name)
        self.count += 1


In [3]:
bot = Bot()

In [4]:
bot.read('https://habr.com/ru/users/Sberbank/')

2

In [5]:
template = cv2.imread('logo.png')
template = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
template = cv2.resize(template,(20, 20))
(tH, tW) = template.shape[:2]
#cv2.imshow("Template", template)
#cv2.waitKey()

In [10]:
#image = cv2.imread("habr.png")
#image = cv2.imread("sber_opros_screenshot.png")
#image = cv2.imread("sber.png")
name = str(bot.count-1) + '.png'
image = cv2.imread(name)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
found = None

In [11]:
for scale in np.linspace(0.2, 1.0, 20)[::-1]:
    resized = imutils.resize(gray, width = int(gray.shape[1] * scale))
    r = gray.shape[1] / float(resized.shape[1])
 
    if resized.shape[0] < tH or resized.shape[1] < tW:
        break
    
    edged = cv2.Canny(resized, 50, 200)
    result = cv2.matchTemplate(edged, template, cv2.TM_CCOEFF)
    (_, maxVal, _, maxLoc) = cv2.minMaxLoc(result)

    clone = np.dstack([edged, edged, edged])
    cv2.rectangle(clone, (maxLoc[0], maxLoc[1]),
    (maxLoc[0] + tW, maxLoc[1] + tH), (0, 0, 255), 2)
    #cv2.imshow("Visualize", clone)
    #cv2.waitKey(0)
    print(maxVal)

    if found is None or maxVal > found[0]:
        found = (maxVal, maxLoc, r)

1559863.375
1392708.0
1420301.75
1122663.0
1071863.75
1063301.75
1084289.375
1122431.0
1053040.375
1217739.125
1080425.5
1014712.5625
916785.5625
900156.375
1002156.375
908347.625
917094.25
861565.4375
994297.6875
887977.6875


In [12]:
print(found[0])
if found[0] < 1200000.0:
    print("NOT SBERBANK LOGO")
    
(_, maxLoc, r) = found
(startX, startY) = (int(maxLoc[0] * r), int(maxLoc[1] * r))
(endX, endY) = (int((maxLoc[0] + tW) * r), int((maxLoc[1] + tH) * r))
 
cv2.rectangle(image, (startX, startY), (endX, endY), (0, 0, 255), 2)
cv2.imshow("ImageDetected", image)
cv2.waitKey(0)

1559863.375


-1

In [32]:
t = '''xn----7sbcarcdzob4a7age2a.xn--p1ai/,sber-bonus.club/,sb-on-line.ru/,opros-2019.com,sber-opros.pro'''.split(',')
t

['xn----7sbcarcdzob4a7age2a.xn--p1ai/',
 'sber-bonus.club/',
 'sb-on-line.ru/',
 'opros-2019.com',
 'sber-opros.pro']